In [1]:
import pandas as pd
import re
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import readability


In [2]:
list_=[]
df = pd.read_csv('forumdata/doctorslounge_data.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_0.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_1.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_2.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_3.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_4.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/ehealthforum_data_5.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/webmd_data_0.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/webmd_data_1.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/webmd_data_2.csv', index_col=None, header=0)
list_.append(df)
df = pd.read_csv('forumdata/scarleteen_data.csv', index_col=None, header=0)
list_.append(df)

In [3]:
df=pd.concat(list_,ignore_index=True)
df=df.drop([df.columns[0],'response_time'],1)
df=df.drop_duplicates()

In [4]:
posts=df.shape[0]
convos=df['post'].unique().shape[0]
print "Forum dataset consists of %s posts across %s conversations" % (posts,convos)

Forum dataset consists of 145255 posts across 63846 conversations


In [7]:
df.groupby('response_type').count()

,post,response,subject,user_name,user_profile
response_type,,,,,
asker,63846,63846,63846,63846,63846
doctor,50699,50699,50699,50699,50699
other,30404,30404,30404,30397,30404
staff,306,306,306,306,306


In [32]:
labeled = 1-df.groupby('subject')['response'].count()[['nevermind','stds']].sum()/float(len(df))
print "%s labeled by subject " % labeled

0.133172696293 labeled by subject 
